# Normalizing Flows

Normalizing flows are for powerful distribution approximation.

For more [detail](https://lilianweng.github.io/posts/2018-10-13-flow-models/).


## Definition

Normalizing flows are for more powerful distribution approximation.
The name “normalizing flow” can be interpreted as the following:

1. “Normalizing” means that the change of variables gives a normalized density after applying an invertible transformation.
2. “Flow” means that the invertible transformations can be composed with each other to create more complex invertible transformations.

In normalizing flows, we wish to map simple distributions (easy to sample and evaluate densities) to complex ones (learned via data)by applying a sequence of invertible transformation functions. Flowing through a chain of transformations, we repeatedly substitute the variable for the new one according to the change of variables theorem and eventually obtain a probability distribution of the final target variable.

<img src="notebook-images/normalizing-flow.png" alt="Alt Text" width="900"/>


*Figure 1: A representation of normalizing flows, illustrating the transformation of a simple, easy-to-sample distribution into a complex target distribution. This is achieved by applying a sequence of invertible transformations, where each step leverages the change of variables theorem to update the density of the transformed variable progressively toward the desired target distribution.*




## Change of Variables Formula

The change of variables formula describes how to evaluate the densities of a random variable that is a deterministic transformation from another variable.
**Change of Variables:** 

**Change of Variables:** Let _Z_ and _X_ be random variables that are related by a mapping  
$f: \mathbb{R}^n \rightarrow \mathbb{R}^n$ such that $X = f(Z)$ and $Z = f^{-1}(X)$. Then:

$$
p_X(\mathbf{x}) = p_Z\left(f^{-1}(\mathbf{x})\right) \left|\det\left(\frac{\partial f^{-1}(\mathbf{x})}{\partial \mathbf{x}}\right)\right|
$$

*  The input and output dimensions must be the same.
*  The transformation must be invertible.
*  Computing the determinant of the Jacobian needs to be efficient and differentiable.


## Training criterion

With normalizing flows in our toolbox, the exact log-likelihood of input data, $\log p(\mathbf{x})$ becomes tractable. As a result, the training criterion of flow-based generative model is simply the negative log-likelihood (NLL) over the training dataset $\mathcal{D}$.
$$\mathcal{L}(\mathcal{D}) = - \frac{1}{\vert\mathcal{D}\vert}\sum_{\mathbf{x} \in \mathcal{D}} \log p(\mathbf{x})$$